In [3]:
import requests
import csv
import pandas as pd
import re

pd.set_option('display.max_columns', None)

# Part 1: Data Extraction

## Football Data API

In [4]:
base_url = "https://api.football-data.org/v4"
pl_seasons_url = f"{base_url}/competitions/PL/matches?season=2022"
pl_teams_url = f"{base_url}/competitions/PL/teams?season=2022"
headers = {"X-Auth-Token": "e956ca357099455ca05a437c205e24e6"} #TODO: get from env variable

### Extract & Format Premier League Team Data

In [40]:
teeam_response = requests.get(pl_teams_url, headers=headers)
pl_team_data = teeam_response.json()

In [42]:
season_response = requests.get(pl_seasons_url, headers=headers)
season_data = season_response.json()

In [63]:
season_data['matches'][1]['utcDate']

'2022-08-06T11:30:00Z'

In [47]:
season_data_df = pd.DataFrame(season_data['matches'])

In [49]:
req_cols = ['id', 'utcDate', 'status', 'homeTeam', 'awayTeam', 'score']

In [51]:
season_data_df = season_data_df[req_cols]
season_data_df.head(2)

,id,utcDate,status,homeTeam,awayTeam,score
0,416384,2022-08-05T19:00:00Z,FINISHED,"{'id': 354, 'name': 'Crystal Palace FC', 'shor...","{'id': 57, 'name': 'Arsenal FC', 'shortName': ...","{'winner': 'AWAY_TEAM', 'duration': 'REGULAR',..."
1,416383,2022-08-06T11:30:00Z,FINISHED,"{'id': 63, 'name': 'Fulham FC', 'shortName': '...","{'id': 64, 'name': 'Liverpool FC', 'shortName'...","{'winner': 'DRAW', 'duration': 'REGULAR', 'ful..."


In [56]:
home_team = pd.json_normalize(season_data_df['homeTeam']).add_prefix('home_team_')
away_team = pd.json_normalize(season_data_df['awayTeam']).add_prefix('away_team_')
score = pd.json_normalize(season_data_df['score']).add_prefix('score_')

In [57]:
score

,score_winner,score_duration,score_fullTime.home,score_fullTime.away,score_halfTime.home,score_halfTime.away
0,AWAY_TEAM,REGULAR,0,2,0,1
1,DRAW,REGULAR,2,2,1,0
2,HOME_TEAM,REGULAR,4,1,2,1
3,HOME_TEAM,REGULAR,2,0,0,0
4,HOME_TEAM,REGULAR,2,1,1,1
...,...,...,...,...,...,...
375,DRAW,REGULAR,1,1,0,1
376,DRAW,REGULAR,1,1,1,1
377,HOME_TEAM,REGULAR,1,0,0,0
378,HOME_TEAM,REGULAR,2,1,2,1


In [6]:
pl_teams = [
    {
    "id": team["id"], 
    "name": team["name"],
    "shortName": team["shortName"],
    "address": team["address"]
    } 
    for team in pl_team_data["teams"]
]

In [7]:
pl_teams_df = pd.read_csv("resources/data/pl_teams.csv")

In [8]:
set(pl_teams_df['city'])

{'Birmingham',
 'Bournemouth',
 'Brentford',
 'Brighton & Hove',
 'Leeds',
 'Leicester',
 'Liverpool',
 'London',
 'Manchester',
 'Newcastle upon Tyne',
 'Nottingham',
 'Southampton',
 'Wolverhampton'}

In [9]:
import urllib.parse

cities = {'Birmingham', 'Bournemouth', 'Brentford', 'Brighton & Hove', 'Leeds', 'Leicester', 
          'Liverpool', 'London', 'Manchester', 'Newcastle upon Tyne', 'Nottingham', 'Southampton', 'Wolverhampton'}

encoded_cities = {urllib.parse.quote(city) for city in cities}

print(encoded_cities)

{'Birmingham', 'Liverpool', 'Brighton%20%26%20Hove', 'Leeds', 'Newcastle%20upon%20Tyne', 'London', 'Manchester', 'Bournemouth', 'Southampton', 'Nottingham', 'Wolverhampton', 'Leicester', 'Brentford'}


In [10]:
unique_cities = set(pd.read_csv("resources/data/pl_teams.csv")["city"])

In [11]:
unique_cities = {urllib.parse.quote(city) for city in unique_cities}

In [12]:
unique_cities

{'Birmingham',
 'Bournemouth',
 'Brentford',
 'Brighton%20%26%20Hove',
 'Leeds',
 'Leicester',
 'Liverpool',
 'London',
 'Manchester',
 'Newcastle%20upon%20Tyne',
 'Nottingham',
 'Southampton',
 'Wolverhampton'}

# Weather data

In [13]:
START_DATE = "2022-05-01"
END_DATE = "2023-06-01"

BASE = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline"
TIME = f"{START_DATE}/{END_DATE}"
PARAMS = "?unitGroup=metric&key=NKSZZCF5RF5HEK47WQWRA8NC8&contentType=json"

In [39]:
for city in unique_cities:
    BASE_URL = f"{BASE}/{city}/{TIME}?{PARAMS}"
#     requests.get(BASE_URL)

In [14]:
url = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/Newcastle%20upon%20Tyne/2022-05-01/2023-06-01?unitGroup=metric&key=X8XBGA475B5W32MGBLXS8KEYW&contentType=json"

In [15]:
res = requests.get(url)

In [16]:
data = res.json()

In [18]:
data.keys()

dict_keys(['queryCost', 'latitude', 'longitude', 'resolvedAddress', 'address', 'timezone', 'tzoffset', 'days', 'stations'])

In [19]:
days_df = pd.DataFrame(data['days'])

In [20]:
days_df

,datetime,datetimeEpoch,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,precip,precipprob,precipcover,preciptype,snow,snowdepth,windgust,windspeed,winddir,pressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,severerisk,sunrise,sunriseEpoch,sunset,sunsetEpoch,moonphase,conditions,description,icon,stations,source,hours,tzoffset
0,2022-05-01,1651359600,14.7,8.7,11.5,14.7,7.7,11.3,8.5,82.4,5.678,100.0,33.33,[rain],0.0,0.0,14.8,8.6,184.7,1020.0,70.4,17.9,141.2,12.1,5.0,10.0,05:26:41,1651379201,20:41:46,1651434106,0.02,"Rain, Partially cloudy",Partly cloudy throughout the day with a chance...,rain,"[03275099999, 03243399999, EGNT, D7177, 032400...",obs,"[{'datetime': '00:00:00', 'datetimeEpoch': 165...",NaN
1,2022-05-02,1651446000,11.6,7.8,10.1,11.6,6.9,9.5,7.1,82.5,0.106,100.0,20.83,[rain],0.0,0.0,24.3,14.8,9.1,1021.9,79.2,13.9,65.5,5.6,2.0,10.0,05:24:33,1651465473,20:43:40,1651520620,0.05,"Rain, Partially cloudy",Partly cloudy throughout the day with rain.,rain,"[03275099999, 03243399999, EGNT, D7177, 032400...",obs,"[{'datetime': '00:00:00', 'datetimeEpoch': 165...",NaN
2,2022-05-03,1651532400,11.0,6.9,8.9,11.0,6.0,8.4,6.5,85.7,0.703,100.0,20.83,[rain],0.0,0.0,18.0,10.7,132.1,1021.1,81.3,10.0,78.8,6.6,2.0,10.0,05:22:25,1651551745,20:45:35,1651607135,0.09,"Rain, Partially cloudy",Partly cloudy throughout the day with a chance...,rain,"[03275099999, 03243399999, EGNT, D7177, 032400...",obs,"[{'datetime': '00:00:00', 'datetimeEpoch': 165...",NaN
3,2022-05-04,1651618800,16.8,6.2,11.6,16.8,6.2,11.3,8.7,83.4,3.568,100.0,37.50,[rain],0.0,0.0,37.6,22.2,259.3,1015.2,58.5,14.9,168.0,14.4,5.0,10.0,05:20:20,1651638020,20:47:29,1651693649,0.12,"Rain, Partially cloudy",Partly cloudy throughout the day with a chance...,rain,"[03275099999, 03243399999, EGNT, D7177, 032400...",obs,"[{'datetime': '00:00:00', 'datetimeEpoch': 165...",NaN
4,2022-05-05,1651705200,14.9,5.7,11.0,14.9,2.9,10.2,7.8,80.8,0.157,100.0,4.17,[rain],0.0,0.0,31.0,19.3,266.0,1021.4,54.4,18.5,137.3,11.9,6.0,10.0,05:18:15,1651724295,20:49:23,1651780163,0.15,"Rain, Partially cloudy",Partly cloudy throughout the day with morning ...,rain,"[03275099999, 03243399999, EGNT, D7177, 032400...",obs,"[{'datetime': '00:00:00', 'datetimeEpoch': 165...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392,2023-05-28,1685228400,14.0,6.2,11.0,14.0,6.2,10.7,5.9,72.4,0.000,0.0,0.00,[rain],0.0,0.0,30.9,23.3,62.8,1027.6,55.0,12.2,172.1,14.8,7.0,10.0,04:40:04,1685245204,21:28:22,1685305702,0.28,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"[03243399999, 03275099999, EGNT, D7177, 032300...",obs,"[{'datetime': '00:00:00', 'datetimeEpoch': 168...",NaN
393,2023-05-29,1685314800,17.0,3.0,10.8,17.0,3.0,10.8,5.4,72.8,0.000,0.0,0.00,None,0.0,0.0,24.5,13.0,98.6,1031.1,5.3,12.8,303.5,26.1,8.0,10.0,04:38:53,1685331533,21:29:46,1685392186,0.32,Clear,Clear conditions throughout the day.,clear-day,"[03243399999, 03275099999, EGNT, D7177, 032300...",obs,"[{'datetime': '00:00:00', 'datetimeEpoch': 168...",NaN
394,2023-05-30,1685401200,15.1,4.2,10.5,15.1,4.2,10.5,7.2,80.7,0.000,0.0,0.00,None,0.0,0.0,22.0,12.9,69.3,1033.1,49.7,12.4,185.2,16.0,6.0,10.0,04:37:45,1685417865,21:31:09,1685478669,0.35,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"[03243399999, 03275099999, EGNT, D7177, 032300...",obs,"[{'datetime': '00:00:00', 'datetimeEpoch': 168...",NaN
395,2023-05-31,1685487600,13.1,9.9,11.3,13.1,8.5,11.2,7.6,78.1,0.000,0.0,0.00,None,0.0,0.0,20.9,14.4,65.5,1031.5,88.1,13.8,124.0,10.6,3.0,10.0,04:36:40,1685504200,21:32:29,1685565149,0.38,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"[03243399999, 03275099999, EGNT, D7177, 032300...",obs,"[{'datetime': '00:00:00', 'datetimeEpoch': 168...",NaN


In [21]:
required_cols = ["datetime", "temp", "humidity", "precip", "windspeed"]

In [28]:
new_df = days_df[required_cols]

In [29]:
new_df

,datetime,temp,humidity,precip,windspeed
0,2022-05-01,11.5,82.4,5.678,8.6
1,2022-05-02,10.1,82.5,0.106,14.8
2,2022-05-03,8.9,85.7,0.703,10.7
3,2022-05-04,11.6,83.4,3.568,22.2
4,2022-05-05,11.0,80.8,0.157,19.3
...,...,...,...,...,...
392,2023-05-28,11.0,72.4,0.000,23.3
393,2023-05-29,10.8,72.8,0.000,13.0
394,2023-05-30,10.5,80.7,0.000,12.9
395,2023-05-31,11.3,78.1,0.000,14.4


In [25]:
urllib.parse.unquote("Newcastle%20upon%20Tyne")

'Newcastle upon Tyne'

In [37]:
new_df.loc[:, 'city'] = urllib.parse.unquote("Newcastle%20upon%20Tyne")

In [38]:
new_df

,datetime,temp,humidity,precip,windspeed,city
0,2022-05-01,11.5,82.4,5.678,8.6,Newcastle upon Tyne
1,2022-05-02,10.1,82.5,0.106,14.8,Newcastle upon Tyne
2,2022-05-03,8.9,85.7,0.703,10.7,Newcastle upon Tyne
3,2022-05-04,11.6,83.4,3.568,22.2,Newcastle upon Tyne
4,2022-05-05,11.0,80.8,0.157,19.3,Newcastle upon Tyne
...,...,...,...,...,...,...
392,2023-05-28,11.0,72.4,0.000,23.3,Newcastle upon Tyne
393,2023-05-29,10.8,72.8,0.000,13.0,Newcastle upon Tyne
394,2023-05-30,10.5,80.7,0.000,12.9,Newcastle upon Tyne
395,2023-05-31,11.3,78.1,0.000,14.4,Newcastle upon Tyne


In [39]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397 entries, 0 to 396
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   datetime   397 non-null    object 
 1   temp       397 non-null    float64
 2   humidity   397 non-null    float64
 3   precip     397 non-null    float64
 4   windspeed  397 non-null    float64
 5   city       397 non-null    object 
dtypes: float64(4), object(2)
memory usage: 18.7+ KB
